# Activity 2: K-nn classification with credit data 

In this activity, we take the German Credit Data and use it to perform K Nearest Neighbors classification. This process involved 
removing NA values, normalization of the data, and experimentation with various sample sizes.

We first load the data from the CSV file and get a summary for the data, after omitting NA values

In [23]:
# install.packages("FNN", dependencies=TRUE, repos='http://cran.us.r-project.org')
library("FNN")
library(dplyr)
library(gdata)

also installing the dependencies 'DEoptimR', 'e1071', 'pcaPP', 'robustbase', 'som', 'lars', 'pls', 'mclust', 'chemometrics', 'mvtnorm'



package 'DEoptimR' successfully unpacked and MD5 sums checked
package 'e1071' successfully unpacked and MD5 sums checked
package 'pcaPP' successfully unpacked and MD5 sums checked
package 'robustbase' successfully unpacked and MD5 sums checked
package 'som' successfully unpacked and MD5 sums checked
package 'lars' successfully unpacked and MD5 sums checked
package 'pls' successfully unpacked and MD5 sums checked
package 'mclust' successfully unpacked and MD5 sums checked
package 'chemometrics' successfully unpacked and MD5 sums checked
package 'mvtnorm' successfully unpacked and MD5 sums checked
package 'FNN' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Siddharth1\AppData\Local\Temp\Rtmp2TrZ3g\downloaded_packages


Warning message:
"package 'FNN' was built under R version 3.3.3"

In [9]:

data <- read.csv("german_credit_data1.csv")

In [10]:
head(data)

X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
0,67,male,2,own,NA,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,49,male,1,own,little,NA,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,2
5,35,male,1,free,NA,NA,9055,36,education,1


In [11]:
names(data)

[1] "X."               "Age"              "Sex"              "Job"             
 [5] "Housing"          "Saving.accounts"  "Checking.account" "Credit.amount"   
 [9] "Duration"         "Purpose"          "Credit.Risks"

In [13]:
str(data)

'data.frame':	1000 obs. of  11 variables:
 $ X.              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ Age             : int  67 22 49 45 53 35 53 35 61 28 ...
 $ Sex             : Factor w/ 2 levels "female","male": 2 1 2 2 2 2 2 2 2 2 ...
 $ Job             : int  2 2 1 2 2 1 2 3 1 3 ...
 $ Housing         : Factor w/ 3 levels "free","own","rent": 2 2 2 1 1 1 2 3 2 2 ...
 $ Saving.accounts : Factor w/ 4 levels "little","moderate",..: NA 1 1 1 1 NA 3 1 4 1 ...
 $ Checking.account: Factor w/ 3 levels "little","moderate",..: 1 2 NA 1 1 NA NA 2 NA 2 ...
 $ Credit.amount   : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ Duration        : int  6 48 12 42 24 36 24 36 12 30 ...
 $ Purpose         : Factor w/ 8 levels "business","car",..: 6 6 4 5 2 4 5 2 6 2 ...
 $ Credit.Risks    : int  1 2 1 1 2 1 1 1 1 2 ...


In [16]:
data <- na.omit(data)

In [34]:
str(data)
summary(data)

'data.frame':	522 obs. of  11 variables:
 $ X.              : int  1 3 4 7 9 10 11 12 13 14 ...
 $ Age             : int  22 45 53 35 28 25 24 22 60 28 ...
 $ Sex             : Factor w/ 2 levels "female","male": 1 2 2 2 2 1 1 1 2 1 ...
 $ Job             : int  2 2 2 3 3 2 2 2 1 2 ...
 $ Housing         : Factor w/ 3 levels "free","own","rent": 2 1 1 3 2 3 3 2 2 3 ...
 $ Saving.accounts : Factor w/ 4 levels "little","moderate",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Checking.account: Factor w/ 3 levels "little","moderate",..: 2 1 1 2 2 2 1 2 1 1 ...
 $ Credit.amount   : int  5951 7882 4870 6948 5234 1295 4308 1567 1199 1403 ...
 $ Duration        : int  48 42 24 36 30 12 48 12 24 15 ...
 $ Purpose         : Factor w/ 8 levels "business","car",..: 6 5 2 2 2 2 1 6 2 2 ...
 $ Credit.Risks    : int  2 1 2 1 2 2 2 1 2 1 ...
 - attr(*, "na.action")=Class 'omit'  Named int [1:478] 1 3 6 7 9 17 18 20 21 25 ...
  .. ..- attr(*, "names")= chr [1:478] "1" "3" "6" "7" ...


       X.             Age            Sex           Job        Housing   
 Min.   :  1.0   Min.   :19.00   female:168   Min.   :0.000   free: 65  
 1st Qu.:233.5   1st Qu.:26.00   male  :354   1st Qu.:2.000   own :349  
 Median :502.5   Median :31.50                Median :2.000   rent:108  
 Mean   :493.5   Mean   :34.89                Mean   :1.875             
 3rd Qu.:736.8   3rd Qu.:41.00                3rd Qu.:2.000             
 Max.   :999.0   Max.   :75.00                Max.   :3.000             
                                                                        
   Saving.accounts Checking.account Credit.amount      Duration    
 little    :412    little  :245     Min.   :  276   Min.   : 6.00  
 moderate  : 64    moderate:224     1st Qu.: 1298   1st Qu.:12.00  
 quite rich: 23    rich    : 53     Median : 2326   Median :18.00  
 rich      : 23                     Mean   : 3279   Mean   :21.34  
                                    3rd Qu.: 3971   3rd Qu.:26.75  
        

We now normalize the data in the various numeric columns that we have. I tried two normalization techniques, min-max normalization and scaling. Scaling gave better results and hence I persisted with scaling in the end. 

In [57]:
# http://vitalflux.com/data-science-scale-normalize-numeric-data-using-r/
# Function for min-max normalization
normalize <- function(x) {
    return ((x - min(x)) / (max(x) - min(x)))
  }

In [119]:
# Applying scaling based normalization to the data

data1 <- data
# column <- sapply(data1,as.numeric)
column<-sapply(data1,is.numeric)
data2 <- data1
data2[column] <- as.data.frame( scale(data1[column] ))
head(data2)



,X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
2,-1.691757,-1.093398286,female,0.1823654,own,little,moderate,0.9122954,2.1373056,radio/TV,1.1213059
4,-1.684887,0.857752104,male,0.1823654,free,little,little,1.5715298,1.6563082,furniture/equipment,-0.8901088
5,-1.681453,1.536413109,male,0.1823654,free,little,little,0.5432470,0.2133159,car,1.1213059
8,-1.671148,0.009425847,male,1.6469000,rent,little,moderate,1.2526666,1.1753108,car,-0.8901088
10,-1.664278,-0.584402532,male,1.6469000,own,little,moderate,0.6675149,0.6943133,car,1.1213059
11,-1.660843,-0.838900409,female,0.1823654,rent,little,moderate,-0.6772414,-0.7486789,car,1.1213059


Now we sample the data, for various sampling rates (0.9,0.8,0.7) etc, and use all the other columns except Credit.Risks for predicting the labels. The result is stored in a dataframe, and also written to a CSV file at the end.

In [214]:
n.points <- 522 # number of rows in the dataset
sampling.rate <- 0.9
num.test.set.labels <- n.points * (1 - sampling.rate)
training <- sample(1:n.points, sampling.rate * n.points,replace=FALSE)
train <- subset(data2[training, ], select = c(Age,factor(Sex), factor(Checking.account),factor(Housing), factor(Saving.accounts),factor(Purpose) ,
 Job,  Credit.amount, Duration ))

# train <- subset(data1[training, ], select = c(Age,  factor(Saving.accounts), factor(Checking.account), factor(Housing)))
testing <- setdiff(1:n.points, training)
# test <- subset(data[testing, ], select = c(Age, Sex, Saving.accounts, Checking.account, Credit.amount, Duration, Purpose, Credit.Risks))
test <- subset(data2[testing, ], select = c(Age,factor(Sex),  factor(Checking.account), factor(Housing),factor(Saving.accounts) ,factor(Purpose) ,
Job, Credit.amount, Duration))
cl <- data2$Credit.Risks[training]
true.labels <- data2$Credit.Risks[testing]

In [215]:
d <- data.frame()

for (k in 1:20) {
print(k)
predicted.labels <- knn(train, test, cl, k)
# We're using the R function knn()
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /
num.test.set.labels
print(misclassification.rate)
d <- rbind(d,c(k, misclassification.rate))
}

[1] 1
[1] 0.4597701
[1] 2
[1] 0.4214559
[1] 3
[1] 0.4214559
[1] 4
[1] 0.3831418
[1] 5
[1] 0.3639847
[1] 6
[1] 0.3639847
[1] 7
[1] 0.3448276
[1] 8
[1] 0.3639847
[1] 9
[1] 0.3256705
[1] 10
[1] 0.3831418
[1] 11
[1] 0.3448276
[1] 12
[1] 0.3831418
[1] 13
[1] 0.3831418
[1] 14
[1] 0.3639847
[1] 15
[1] 0.3639847
[1] 16
[1] 0.3256705
[1] 17
[1] 0.2873563
[1] 18
[1] 0.3256705
[1] 19
[1] 0.2681992
[1] 20
[1] 0.3448276


In [216]:
d

X1,X0.459770114942529
1,0.4597701
2,0.4214559
3,0.4214559
4,0.3831418
5,0.3639847
6,0.3639847
7,0.3448276
8,0.3639847
9,0.3256705
10,0.3831418


In [217]:
d09 <- d

In [226]:
n.points <- 522 # number of rows in the dataset
sampling.rate <- 0.8
num.test.set.labels <- n.points * (1 - sampling.rate)
training <- sample(1:n.points, sampling.rate * n.points,replace=FALSE)
train <- subset(data2[training, ], select = c(Age,factor(Sex), factor(Checking.account),factor(Housing), factor(Saving.accounts),factor(Purpose) ,
 Job,  Credit.amount, Duration ))

# train <- subset(data1[training, ], select = c(Age,  factor(Saving.accounts), factor(Checking.account), factor(Housing)))
testing <- setdiff(1:n.points, training)
# test <- subset(data[testing, ], select = c(Age, Sex, Saving.accounts, Checking.account, Credit.amount, Duration, Purpose, Credit.Risks))
test <- subset(data2[testing, ], select = c(Age,factor(Sex),  factor(Checking.account), factor(Housing),factor(Saving.accounts) ,factor(Purpose) ,
Job, Credit.amount, Duration))
cl <- data2$Credit.Risks[training]
true.labels <- data2$Credit.Risks[testing]

In [227]:
d08 <- data.frame()

for (k in 1:20) {
print(k)
predicted.labels <- knn(train, test, cl, k)
# We're using the R function knn()
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /
num.test.set.labels
print(misclassification.rate)
d08 <- rbind(d08,c(k, misclassification.rate))
}

[1] 1
[1] 0.3256705
[1] 2
[1] 0.3544061
[1] 3
[1] 0.3544061
[1] 4
[1] 0.3639847
[1] 5
[1] 0.316092
[1] 6
[1] 0.3065134
[1] 7
[1] 0.2586207
[1] 8
[1] 0.2586207
[1] 9
[1] 0.2969349
[1] 10
[1] 0.2298851
[1] 11
[1] 0.2873563
[1] 12
[1] 0.2490421
[1] 13
[1] 0.2586207
[1] 14
[1] 0.2298851
[1] 15
[1] 0.2203065
[1] 16
[1] 0.2586207
[1] 17
[1] 0.2681992
[1] 18
[1] 0.2777778
[1] 19
[1] 0.2681992
[1] 20
[1] 0.316092


In [228]:
d08

X1,X0.325670498084291
1,0.3256705
2,0.3544061
3,0.3544061
4,0.3639847
5,0.3160920
6,0.3065134
7,0.2586207
8,0.2586207
9,0.2969349
10,0.2298851


In [324]:
n.points <- 522 # number of rows in the dataset
sampling.rate <- 0.7
num.test.set.labels <- n.points * (1 - sampling.rate)
training <- sample(1:n.points, sampling.rate * n.points,replace=FALSE)
train <- subset(data2[training, ], select = c(Age,factor(Sex), factor(Checking.account),factor(Housing), factor(Saving.accounts),factor(Purpose) ,
 Job,  Credit.amount, Duration ))

# train <- subset(data1[training, ], select = c(Age,  factor(Saving.accounts), factor(Checking.account), factor(Housing)))
testing <- setdiff(1:n.points, training)
# test <- subset(data[testing, ], select = c(Age, Sex, Saving.accounts, Checking.account, Credit.amount, Duration, Purpose, Credit.Risks))
test <- subset(data2[testing, ], select = c(Age,factor(Sex),  factor(Checking.account), factor(Housing),factor(Saving.accounts) ,factor(Purpose) ,
Job, Credit.amount, Duration))
cl <- data2$Credit.Risks[training]
true.labels <- data2$Credit.Risks[testing]

In [325]:
d07 <- data.frame()

for (k in 1:20) {
print(k)
predicted.labels <- knn(train, test, cl, k)
# We're using the R function knn()
num.incorrect.labels <- sum(predicted.labels != true.labels)
misclassification.rate <- num.incorrect.labels /
num.test.set.labels
print(misclassification.rate)
d07 <- rbind(d07,data.frame(k, misclassification.rate))
}

[1] 1
[1] 0.4469987
[1] 2
[1] 0.4342273
[1] 3
[1] 0.3959132
[1] 4
[1] 0.4214559
[1] 5
[1] 0.3767561
[1] 6
[1] 0.3639847
[1] 7
[1] 0.3703704
[1] 8
[1] 0.3448276
[1] 9
[1] 0.357599
[1] 10
[1] 0.3448276
[1] 11
[1] 0.293742
[1] 12
[1] 0.3320562
[1] 13
[1] 0.3448276
[1] 14
[1] 0.3384419
[1] 15
[1] 0.3639847
[1] 16
[1] 0.3895275
[1] 17
[1] 0.3639847
[1] 18
[1] 0.4022989
[1] 19
[1] 0.3831418
[1] 20
[1] 0.4086845


In [326]:
d07copy <- d07

In [328]:
datafinal <- merge(d09, d08, by="X1")

In [332]:
datafinal <-  rename.vars(datafinal, c("X1"), c("k"))


gdata: Unable to locate valid perl interpreter
gdata: 
gdata: read.xls() will be unable to read Excel XLS and XLSX files
gdata: unless the 'perl=' argument is used to specify the location of a
gdata: valid perl intrpreter.
gdata: 
gdata: (To avoid display of this message in the future, please ensure
gdata: perl is installed and available on the executable search path.)
gdata: Unable to load perl libaries needed by read.xls()
gdata: to support 'XLX' (Excel 97-2004) files.

gdata: Unable to load perl libaries needed by read.xls()
gdata: to support 'XLSX' (Excel 2007+) files.

gdata: Run the function 'installXLSXsupport()'
gdata: to automatically download and install the perl
gdata: libaries needed to support Excel XLS and XLSX formats.

Attaching package: 'gdata'

The following objects are masked from 'package:dplyr':

    combine, first, last

The following object is masked from 'package:stats':

    nobs

The following object is masked from 'package:utils':

    object.size

The follow


Changing in datafinal        
From: X1
To:   k 



In [333]:
datafinal <- merge(datafinal, d07, by="k")

In [335]:
datafinal <-  rename.vars(datafinal, c("X0.459770114942529","X0.325670498084291","misclassification.rate"), 
                                     c("train=90","train=80","train=70"))


Changing in datafinal                                                                  
From: X0.459770114942529 X0.325670498084291 misclassification.rate
To:   train=90           train=80           train=70              



In [336]:
datafinal

k,train=90,train=80,train=70
1,0.4597701,0.3256705,0.4469987
2,0.4214559,0.3544061,0.4342273
3,0.4214559,0.3544061,0.3959132
4,0.3831418,0.3639847,0.4214559
5,0.3639847,0.3160920,0.3767561
6,0.3639847,0.3065134,0.3639847
7,0.3448276,0.2586207,0.3703704
8,0.3639847,0.2586207,0.3448276
9,0.3256705,0.2969349,0.3575990
10,0.3831418,0.2298851,0.3448276


In [337]:
# write.csv(datafinal,file="knnresults.csv",row.names=FALSE)

In [2]:
results <- read.csv(file="knnresults.csv", header = TRUE)

In [3]:
results

k,train.90,train.80,train.70
1,0.4597701,0.3256705,0.4469987
2,0.4214559,0.3544061,0.4342273
3,0.4214559,0.3544061,0.3959132
4,0.3831418,0.3639847,0.4214559
5,0.3639847,0.3160920,0.3767561
6,0.3639847,0.3065134,0.3639847
7,0.3448276,0.2586207,0.3703704
8,0.3639847,0.2586207,0.3448276
9,0.3256705,0.2969349,0.3575990
10,0.3831418,0.2298851,0.3448276
